In [1]:
import logging

logging.basicConfig(level=logging.INFO)


In [2]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
from torch.optim import SGD, Adam
from torch.nn import CrossEntropyLoss, MSELoss

# from pennylane import NesterovMomentumOptimizer

from qcc.ml.data import Data, BinaryData
from qcc.ml.optimize import Optimizer
from qcc.ml.data import ImageTransform, ImageTransform1D, ClassicalImageTransform
from qcc.experiment import Experiment
from qcc.ml.model import Model

from pathlib import Path
from qcc.file import new_dir

# from qcc.quantum.pennylane.ansatz import MQCCOptimized # as Module
# from qcc.quantum.pennylane.ansatz import QCNN as Module
# from qcc.ml.cnn import ConvolutionalNeuralNetwork as Module
# from qcc.ml.mlp import MultiLayerPerceptron as Module
from qcc.quantum.quanvolution import QuanvolutionalNeuralNetwork as Module
# from qcc.ml.hybrid_cnn import MQCCNonHybrid as Module

# from qcc.quantum.pennylane.ansatz.basic import BasicFiltering6
from qcc.quantum.pennylane.pyramid import Pyramid
from qcc.quantum.pennylane.c2q import ConvolutionAngleFilter, ConvolutionComplexAngleFilter, ConvolutionFilter
from qcc.quantum.pennylane.local import define_filter
from qcc.ml.hybrid_cnn import AnsatzFilter


In [3]:
# Meta parameters
name = "sdfljhvosdv"
filename = Path(f"results/{name}")
num_trials = 1
silent = False
is_quantum = False

# Ansatz parameters
dims = (16, 16, 3)
num_layers = 4
module_options = {
    # "U_filter": ConvolutionAngleFilter,
    # "pre_op": True,
    # "num_features": 4,
    # "U_fully_connected": ConvolutionAngleFilter,
    # "pooling": True,
    # "filter_shape": (2, 2),
    # "q2c_method": "parity"
    # "bias": False
    # "ansatz": MQCCOptimized
}


In [4]:
filename = new_dir(filename, overwrite=True)
filename = filename / name

if is_quantum:
    module = Module.from_dims
else:
    module = Module

# Create module
module = module(
    dims,
    num_layers=num_layers,
    **module_options,
)
data = BinaryData(
    CIFAR10,
    ImageTransform(dims) if is_quantum else ClassicalImageTransform(dims),
    batch_size=(8,8),
)
optimizer = Optimizer(Adam)
loss = CrossEntropyLoss()
model = Model.with_logging(module, data, optimizer, loss, epoch=8)

# Log important values
model.logger.info(f"Circuit ID: {name}")
model.logger.info(f"{module=}")
model.logger.info(f"{data=}")
model.logger.info(f"{optimizer=}")
model.logger.info(f"{loss=}")

model.logger.info(f"{num_trials=}")
model.logger.info(f"{dims=}")
model.logger.info(f"{num_layers=}")
# model.logger.info(f"{1=}")
model.logger.info(f"{module_options=}")

# Save circuit drawing
if is_quantum:
    draw_path = filename.with_stem(f"{name}_circuit")
    module.draw(filename=draw_path, decompose=True)


2023-11-20 19:01:53,189: (quanvolutionalneuralnetwork) Circuit ID: sdfljhvosdv
2023-11-20 19:01:53,190: (quanvolutionalneuralnetwork) module=QuanvolutionalNeuralNetwork(
  (0): Quanvolution(
    (qnode): <Quantum Torch Layer: func=circuit>
  )
  (1): ConvolutionalNeuralNetwork(
    (0): Conv2d(12, 1, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1), padding_mode=circular)
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
    (3): Conv2d(1, 1, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1), padding_mode=circular)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): ReLU()
    (6): Conv2d(1, 1, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1), padding_mode=circular)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): ReLU()
    (9): Flatten(start_dim=1, end_dim=-1)
    (10): Linear(in_features=1, out_features=2, bias=True)
  )
)
2023-11-20 19:01:53,190: 

(16, 16, 3) 3


In [5]:
# Run experiment
results_schema = ["accuracy", "training_time", "testing_time"]
experiment = Experiment(model, num_trials, results_schema)
# experiment.partial(silent=silent)
results = experiment(filename=filename)


Files already downloaded and verified
Files already downloaded and verified


2023-11-20 19:01:54,547: (quanvolutionalneuralnetwork) Number of Parameters: 63


KeyboardInterrupt: 

In [ ]:
# Print accuracy results
metrics = ("median", "mean", "max", "min", "std")
for name in results.columns:
    col = results[name]
    msg = (f"{metric}={getattr(col, metric)()}" for metric in metrics)
    msg = ", ".join(msg)
    msg = f"{name}: {msg}"
    model.logger.info(msg)

# Save aggregated loss history figure
display(experiment.draw(filename))
